In [1]:
import os
from openai import OpenAI
import json
from tqdm.notebook import tqdm
import time


In [2]:
client = OpenAI(api_key="your-api-key")


In [4]:
data = []
with open("./data.json", "r") as f:
    for line in f:
        data.append(eval(line))
        
len(data)

74

In [26]:
MODEL_LIMITS = {
    "gpt-3.5-turbo-0125": 16_385,
    "gpt-4-turbo-2024-04-09": 128_000,
    "gpt-4o-2024-05-13": 128_000
}

# The cost per token for each model input.
MODEL_COST_PER_INPUT = {
    "gpt-3.5-turbo-0125": 0.0000005,
    "gpt-4-turbo-2024-04-09": 0.00001,
    "gpt-4o-2024-05-13": 0.000005
}

# The cost per token for each model output.
MODEL_COST_PER_OUTPUT = {
    "gpt-3.5-turbo-0125": 0.0000015,
    "gpt-4-turbo-2024-04-09": 0.00003,
    "gpt-4o-2024-05-13": 0.000015
}



In [33]:
error_cout = 0
total_cost = 0
save_path = "./output_model/"
start = time.time()
model = 'gpt-3.5-turbo-0125'
# model = 'gpt-4o-2024-05-13'
for line in tqdm(data[42:]):
  name = line['name']
  # print(name)
  start = time.time()
  with open(f"./data/task/{name}.txt", "r") as f:
    description = f.read()
  assistant = client.beta.assistants.create(
    instructions="You are a data scientist. I have a data modeling task. You must give me the predicted results as a CSV file as detailed in the following content.  Don't ask me any questions. I provide you with three files. One is training data, one is test data. There is also a sample file for submission",
    model=model,
    tools=[{"type": "code_interpreter"}],
    # tool_resources={
    #   "code_interpreter": {
    #     "file_ids": [train_file.id, test_file.id, sample_file.id]
    #   }
    # }
  )
  files = os.listdir(f"./data_resplit/{name}")
  print("Upload a file with an assistants purpose")
  train_file = client.files.create(
    file=open(f"./data_resplit/{name}/train.csv", "rb"),
    purpose='assistants'
  )
  
  # Upload a file with an "assistants" purpose
  test_file = client.files.create(
    file=open(f"./data_resplit/{name}/test.csv", "rb"),
    purpose='assistants'
  )
  files.remove('test.csv')
  files.remove('train.csv')
  sample_file = client.files.create(
    file=open(f"./data_resplit/{name}/{files[0]}", "rb"),
    purpose='assistants'
  )
  print("start a messages")
  thread = client.beta.threads.create(
    messages=[
      {
        "role": "user",
        "content": description,
        "attachments": [
          {
            "file_id": train_file.id,
            "tools": [{"type": "code_interpreter"}]
          },
          {
            "file_id": test_file.id,
            "tools": [{"type": "code_interpreter"}]
          },
          {
            "file_id": sample_file.id,
            "tools": [{"type": "code_interpreter"}]
          }
        ]
      }
    ]
  )
  print("start client running")
  run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id,
  )
  prompt_tokens = run.usage.prompt_tokens 
  completion_tokens = run.usage.completion_tokens
  cost = 0.03 + run.usage.completion_tokens * MODEL_COST_PER_OUTPUT[model] + run.usage.prompt_tokens * MODEL_COST_PER_INPUT[model]

  print("read  messages")
  # cost = 0
  if not os.path.exists(f"{save_path}{model}"):
      os.makedirs(f"{save_path}{model}")
  messages = ""
  try:
    messages = client.beta.threads.messages.list(
      thread_id=thread.id
    )
    # print(messages)
    submission_id = messages.data[0].content[0].text.annotations[0].file_path.file_id
  
    with open(f"{save_path}{model}/{name}.csv", "wb") as f:
      f.write(client.files.content(submission_id).read())
    client.files.delete(submission_id)
  except:
    error_cout += 1
    print(f"error {error_cout}")
  total_cost += cost
  with open(f"{save_path}{model}/{name}.json", "w") as f:
    json.dump({"name": name, "model": model, "input": prompt_tokens,
                            "output": completion_tokens, "cost": cost, "time": time.time()-start, 'response': str(messages)}, f)
  with open(f"{save_path}{model}/{name}_message.txt", "w") as f:
    f.write(str(messages))
  print("Total cost: ", total_cost)
  client.files.delete(sample_file.id)
  client.files.delete(train_file.id)
  client.files.delete(test_file.id)
  
  client.beta.threads.delete(thread.id)
  client.beta.assistants.delete(assistant.id)
end = time.time()

  0%|          | 0/32 [00:00<?, ?it/s]

Upload a file with an assistants purpose
start a messages
start client running
read  messages
Total cost:  0.129525
Upload a file with an assistants purpose
start a messages
start client running
read  messages
error 1
Total cost:  0.20917
Upload a file with an assistants purpose
start a messages
start client running
read  messages
Total cost:  0.334125
Upload a file with an assistants purpose
start a messages
start client running
read  messages
error 2
Total cost:  0.411325
Upload a file with an assistants purpose
start a messages
start client running
read  messages
Total cost:  0.75716
Upload a file with an assistants purpose
start a messages
start client running
read  messages
error 3
Total cost:  1.322475
Upload a file with an assistants purpose
start a messages
start client running
read  messages
error 4
Total cost:  1.36727
Upload a file with an assistants purpose
start a messages
start client running
read  messages
Total cost:  1.672445
Upload a file with an assistants purpose
st